In [ ]:
#i "nuget: https://pkgs.dev.azure.com/dotnet/TorchSharp/_packaging/Testing%40Local/nuget/v3/index.json"
#r "nuget: TorchSharp-cpu,0.93.7"
using TorchSharp;
using static TorchSharp.TensorExtensionMethods;
using static TorchSharp.torch.distributions;

using Microsoft.DotNet.Interactive.Formatting;
Formatter.SetPreferredMimeTypeFor(typeof(torch.Tensor), "text/plain");
Formatter.Register<torch.Tensor>((torch.Tensor x) => x.ToString(true));

# Random Numbers and Distributions

There is a rich set of random number generation APIs in TorchSharp. We've already seen the ones that are easiest to use: randn(), rand(), and randint(). Normal and uniform distributions are the foundation for many other random number features.

In the `TorchSharp.torch.distributions` static class, there is a much richer collection of distributions. Unlike the three basic generators, these generators are organized as classes that you call a method named `sample()` to get a bunch of random number.

## Setting the seed

Like most random number libraries, TorchSharp allows you to set the seed used for random number generation. One peculiarity about TorchSharp is that using the same initial seed will not lead to the same sequence of numbers when using a CPU vs. a GPU. You cannot reproduce results you had on a CPU by running things on a GPU.

In [ ]:
torch.random.manual_seed(4711);
torch.rand(10).print();
torch.random.manual_seed(17);
torch.rand(10).print();
torch.random.manual_seed(4711);
torch.rand(10).print();

## Coin Toss

For example, to get a single-value sample of the Bernoulli distrubtion, which is a binary false/true, 0/1, yes/no, heads/tails generator, you do the following, passing in the probability of the result being '1':

In [ ]:
var bern = Bernoulli(torch.tensor(0.5f));
bern.sample().item<float>()

The element type of the sample will be determined by the element type of the probability tensor, so using precise number literal syntax is important.

Usually, you want more than one value, you want a tensor-full of them. `sample()` takes as its arguments the size of the dimensions of the tensor.

In [ ]:
bern.sample(3,4)

In [ ]:
var bin = Binomial(torch.tensor(100), torch.tensor(0.25f));
bin.sample().item<float>()

In [ ]:
bin.sample(3,4)

## Categories

In the coin toss scenario, there were two categories -- yes/no, true/false, 0/1, etc. A more general class of distributions support N different categories. The foundational class for that is called 'Categorical,' and it works just like Bernoulli. You tell it how many categories there are, the probabilities for those categories (it doesn't have to be even), and then you get your sample. The length of the probabilities tensor tells the Categorical class how many categories there are. The categories are represented as integers in the range [0..N[.

In [ ]:
var cat = Categorical(torch.tensor(new float[]{0.1f, 0.7f, 0.1f, 0.1f}));
cat.sample(4)

There's a class corresponding to 'Binomial' for categorical distributions. Here, the category is denoted by the index into the tensor. For sample sizes of at least one dimension, the innermost dimension (the last index) representes the category. In other words, each row is a sample, each column is a category. The value in each cell is how many times (out of the total count specified) that the category was selected.

In [ ]:
var mult = Multinomial(100, new float[]{0.1f, 0.7f, 0.1f, 0.1f});
mult.sample().print();
mult.sample(5).print();
mult.sample(2,3)

## Real-valued Distributions

The majority of random distributions are concerned with real numbers, parameteried by either a min/max range, or the mean and standard deviation, or parameters specific to a distribution.

The normal, a.k.a. Gaussian, distribution is the familiar bell-curve, where the likelihood of a value being selected is much higher closer to the mean. 

With 'torch.randn()' and 'torch.rand()', the mean is always zero, and the standard deviation always one. To alter them, you get the sample, multiply by the desired standard deviation, then add the desired mean (in that order).

You can still do that when you are using the distribution classes, but they also allow you to pass in the parameters when creating the distribution class. This is convenient when you are passing the distribution to a function, which doesn't necessarily have to know anything about what kind of distrubition it is given, or its parameters.

In [ ]:
torch.Tensor foo(Distribution dist) { return dist.sample(4,4);}

var norm1 = Normal(torch.tensor(0.5f), torch.tensor(0.125f));
var norm2 = Normal(torch.tensor(0.15f), torch.tensor(0.025f));

foo(norm1).print();
foo(norm2).print();

The same goes for uniformly distributed numbers -- there's a class parameterized by the boundaries: [low,high].

In [ ]:
var uni = Uniform(torch.tensor(10.0f), torch.tensor(17.0f));
foo(uni)